## Transformers

In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd

> There are a lot of transformation we make on datasets. We can create and load our own dataset and then make some transformation on it, for example **loading the data as numpy array and transform it to torch tensors.**

In [15]:
class Wine(Dataset):
    def __init__(self, transform=None):
        xy = pd.read_csv('wine.csv').values
        self.x = xy[:, 1:]
        self.y = xy[:,0:1]
        self.transform = transform
#         print(self.y, self.x.shape)
        
        self.length = self.x.shape[0]

     
    def __getitem__(self, index):
        sample = self.x[index], self.y[index]

        if self.transform:
            sample = self.transform(sample)
        return sample

    def __len__(self):
        return self.length

### Custom Transforms
We are going to use the `__call__` method:
The `__call__` method enables Python programmers to write classes where the instances behave like functions and can be called like a function. When the instance is called as a function; if this method is defined

#### How does the `__call__` works

In [3]:
class Dog:
    def __init__(self):
        pass
    def __call__(self):
        print("Im a dog class")      
e = Dog()
e()
        

Im a dog class


In [4]:
class Dog:
    def __init__(self):
        pass
    def __call__(self, name):
        print("Im a dog {}".format(name))
ob = Dog()
ob("Hello")

Im a dog Hello


### ToTensor Transformation
> we are going to create a transform that converts our numpy array to tensors.

In [22]:
class ToTensor:
    def __call__(self, sample):
        inputs, targets = sample
        return torch.from_numpy(inputs), torch.from_numpy(targets)

In [18]:
wine = Wine()
wine[:2] ## numpy array

(array([[1.423e+01, 1.710e+00, 2.430e+00, 1.560e+01, 1.270e+02, 2.800e+00,
         3.060e+00, 2.800e-01, 2.290e+00, 5.640e+00, 1.040e+00, 3.920e+00,
         1.065e+03],
        [1.320e+01, 1.780e+00, 2.140e+00, 1.120e+01, 1.000e+02, 2.650e+00,
         2.760e+00, 2.600e-01, 1.280e+00, 4.380e+00, 1.050e+00, 3.400e+00,
         1.050e+03]]),
 array([[1.],
        [1.]]))

### Apply ToTensor Transformation to wine

In [24]:
wine = Wine(transform=ToTensor())
wine[0]

(tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
         3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
         1.0650e+03], dtype=torch.float64),
 tensor([1.], dtype=torch.float64))

### MulTransform Transformation
> This transform is very useful, let's say we have an image thatwe want to pass to our Network, Sometimes we may want to scale each pixel value by dividing by `255` we can use this transform.

In [28]:
class MulTransform:
    def __init__(self, factor):
        self.factor = factor
    
    def __call__(self, sample):
        inputs, targets = sample
        inputs *= self.factor
        return inputs, sample

### Using Multiple transforms
> `torchvison` has a `Compose` function that allows us to pass mutiple transform as a list. For example let's say we want to apply transforms on out `Wine` datasets we can do it as follows.

In [29]:
composed = torchvision.transforms.Compose([
    ToTensor(),
    MulTransform(4)
])
composed

Compose(
)

In [30]:
wine = Wine(composed)
wine[:2]

(tensor([[5.6920e+01, 6.8400e+00, 9.7200e+00, 6.2400e+01, 5.0800e+02, 1.1200e+01,
          1.2240e+01, 1.1200e+00, 9.1600e+00, 2.2560e+01, 4.1600e+00, 1.5680e+01,
          4.2600e+03],
         [5.2800e+01, 7.1200e+00, 8.5600e+00, 4.4800e+01, 4.0000e+02, 1.0600e+01,
          1.1040e+01, 1.0400e+00, 5.1200e+00, 1.7520e+01, 4.2000e+00, 1.3600e+01,
          4.2000e+03]], dtype=torch.float64),
 (tensor([[5.6920e+01, 6.8400e+00, 9.7200e+00, 6.2400e+01, 5.0800e+02, 1.1200e+01,
           1.2240e+01, 1.1200e+00, 9.1600e+00, 2.2560e+01, 4.1600e+00, 1.5680e+01,
           4.2600e+03],
          [5.2800e+01, 7.1200e+00, 8.5600e+00, 4.4800e+01, 4.0000e+02, 1.0600e+01,
           1.1040e+01, 1.0400e+00, 5.1200e+00, 1.7520e+01, 4.2000e+00, 1.3600e+01,
           4.2000e+03]], dtype=torch.float64),
  tensor([[1.],
          [1.]], dtype=torch.float64)))

> That's the basics about transforms.